# 1- Environment Setup


## 1.1 - Libraries

In [1]:
# Importação das Bibliotecas
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import graphviz
from plotly.subplots import make_subplots
from collections import Counter
from datetime import datetime
from math import ceil
from scipy.cluster.hierarchy import dendrogram, linkage
import sqlite3
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN, estimate_bandwidth
from sklearn.metrics import silhouette_score, silhouette_samples, pairwise_distances, pairwise_distances_argmin_min, mean_squared_error, f1_score, accuracy_score, precision_score, recall_score, classification_report, r2_score
from sklearn.neighbors import NearestNeighbors
from sklearn.base import clone
from sklearn.manifold import TSNE
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import RFE
import sklearn as sk


## 1.2 - Import Dataset


In [2]:

data_path_train = 
data_path_test = 

df = pd.read_csv(data_path_train)
dft = pd.read_csv(data_path_test)



ModuleNotFoundError: No module named 'google.colab'

In [ ]:
df.head()

,citizen_id,name,title,date_of_birth,city,country,last_year_avg_monthly_charity_donations,environmental_awareness_rating,financial_wellness_index,investment_portfolio_value,...,investments_risk_tolerance,tech_savviness_score,social_media_influence_score,entertainment_engagement_factor,avg_monthly_entertainment_expenses,avg_weekly_exercise_hours,health_consciousness_rating,stress_management_score,overall_well_being,lifestyle_type
0,1,Stanley,Mr.,1992-01-03,Mining City,Data Land,0.0,1.9352,99.6431,9.7221,...,NaN,11.9772,9.9506,1.75750,23.0605,0.1776,0.6504,2.8850,106.282,Travel Enthusiast
1,2,Višeslav,Mr.,2005-03-16,Mining City,Data Land,0.0,0.5672,125.1630,5.4308,...,13.6445,11.8776,5.1521,1.96770,NaN,1.2782,1.0190,3.5777,132.121,Fitness Enthusiast
2,3,Rogerio,Mr.,2005-04-04,Mining City,Data Land,1.0,0.5613,105.2490,15.0829,...,5.8867,14.4247,3.6737,3.69071,37.5970,0.0354,3.0854,5.1906,115.547,Health-Conscious
3,4,Jagat,Mr.,1981-09-25,Mining City,Data Land,0.0,0.3543,95.2219,16.3380,...,9.1074,12.3668,5.5798,1.57690,8.2142,1.7763,0.3677,NaN,101.773,Fitness Enthusiast
4,5,Virgínio,Mr.,1987-07-29,Mining City,Data Land,0.0,0.5943,255.8600,148.2250,...,4.0892,15.8389,3.7712,1.95580,14.3443,0.0000,0.6718,2.7076,264.451,Adventure Seeker


In [ ]:
## Copy the original dataset (so that we can make changes and the original remains intact)
df_original = df.copy()
df.shape

(90165, 21)

## 1.3 - Functions

In [ ]:
## Measure the skewness of a dataframe

def calculate_skewness(df):

    skewness_dict = {}
    for column_name in df.columns:
        skewness = df[column_name].skew()
        skewness_dict[column_name] = skewness
    return skewness_dict

## Measure the kurtosis of a dataframe

def calculate_kurtosis(df):

    kurtosis_dict = {}
    for column_name in df.columns:
        kurtosis = df[column_name].kurtosis()
        kurtosis_dict[column_name] = kurtosis
    return kurtosis_dict


## 1.4 - Data Exploration

In [ ]:
# Display the first few rows of the DataFrame 'df' to get a quick look at the data
df.head()

,citizen_id,name,title,date_of_birth,city,country,last_year_avg_monthly_charity_donations,environmental_awareness_rating,financial_wellness_index,investment_portfolio_value,...,investments_risk_tolerance,tech_savviness_score,social_media_influence_score,entertainment_engagement_factor,avg_monthly_entertainment_expenses,avg_weekly_exercise_hours,health_consciousness_rating,stress_management_score,overall_well_being,lifestyle_type
0,1,Stanley,Mr.,1992-01-03,Mining City,Data Land,0.0,1.9352,99.6431,9.7221,...,NaN,11.9772,9.9506,1.75750,23.0605,0.1776,0.6504,2.8850,106.282,Travel Enthusiast
1,2,Višeslav,Mr.,2005-03-16,Mining City,Data Land,0.0,0.5672,125.1630,5.4308,...,13.6445,11.8776,5.1521,1.96770,NaN,1.2782,1.0190,3.5777,132.121,Fitness Enthusiast
2,3,Rogerio,Mr.,2005-04-04,Mining City,Data Land,1.0,0.5613,105.2490,15.0829,...,5.8867,14.4247,3.6737,3.69071,37.5970,0.0354,3.0854,5.1906,115.547,Health-Conscious
3,4,Jagat,Mr.,1981-09-25,Mining City,Data Land,0.0,0.3543,95.2219,16.3380,...,9.1074,12.3668,5.5798,1.57690,8.2142,1.7763,0.3677,NaN,101.773,Fitness Enthusiast
4,5,Virgínio,Mr.,1987-07-29,Mining City,Data Land,0.0,0.5943,255.8600,148.2250,...,4.0892,15.8389,3.7712,1.95580,14.3443,0.0000,0.6718,2.7076,264.451,Adventure Seeker


In [ ]:
# Provides a concise summary of columns, datatypes, and the presence of null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90165 entries, 0 to 90164
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   citizen_id                               90165 non-null  int64  
 1   name                                     90165 non-null  object 
 2   title                                    90165 non-null  object 
 3   date_of_birth                            90165 non-null  object 
 4   city                                     90165 non-null  object 
 5   country                                  90165 non-null  object 
 6   last_year_avg_monthly_charity_donations  87911 non-null  float64
 7   environmental_awareness_rating           86558 non-null  float64
 8   financial_wellness_index                 88363 non-null  float64
 9   investment_portfolio_value               88362 non-null  float64
 10  investments_risk_appetite                88813

In [ ]:
# df.describe produces a statistical summary of all numerical columns. This was important for us to detect the both 'last_year_avg_charity' and 'weekly_exercise_hours'
# have negative values

df.describe().T

,count,mean,std,min,25%,50%,75%,max
citizen_id,90165.0,45083.000000,26028.537848,1.0000,22542.000000,45083.00000,67624.000000,90165.00000
last_year_avg_monthly_charity_donations,87911.0,0.196324,0.554079,-3.0000,0.000000,0.00000,0.000000,5.00000
environmental_awareness_rating,86558.0,1.010520,0.914724,0.0000,0.563925,0.66080,0.972075,12.83600
financial_wellness_index,88363.0,105.308175,50.667686,0.0500,94.472800,99.89540,106.251000,593.84700
investment_portfolio_value,88362.0,19.109577,35.481222,0.5000,5.718225,10.48305,15.438725,446.86600
investments_risk_appetite,88813.0,4.443178,2.677584,0.0000,1.902400,4.93970,6.913400,12.07270
investments_risk_tolerance,87009.0,7.151049,3.945640,0.0000,4.659300,6.08060,8.391300,44.25620
tech_savviness_score,87009.0,13.459303,1.486830,6.8615,12.495200,13.47790,14.441400,30.43360
social_media_influence_score,87911.0,6.375599,4.258794,0.0000,3.340800,4.80220,8.807200,44.96590
entertainment_engagement_factor,89263.0,1.841974,0.343801,0.2804,1.643000,1.86930,2.070800,3.69071


In [ ]:
# Provides staistics for categorical variables (w/ datatype Object)
df.describe(include = ['O'])

,name,title,date_of_birth,city,country,lifestyle_type
count,90165,90165,90165,90165,90165,90165
unique,9946,4,16878,1,1,5
top,Emma,Mr.,1994-05-29,Mining City,Data Land,Health-Conscious
freq,246,45126,20,90165,90165,18131


# 2 - Train Set Preparation

### 2.0 - Determine initial metric and non_metric variables

In [ ]:
#We define our variables as either metric or non_metric and assigned them to their repestive list.

metric = ['last_year_avg_monthly_charity_donations',
'environmental_awareness_rating',
'financial_wellness_index',
'investment_portfolio_value',
'investments_risk_appetite',
'investments_risk_tolerance',
'tech_savviness_score',
'social_media_influence_score',
'entertainment_engagement_factor',
'avg_monthly_entertainment_expenses',
'avg_weekly_exercise_hours',
'health_consciousness_rating',
'stress_management_score',
'overall_well_being',
'age',
'female']


non_metric = ['title',
'date_of_birth']


## 2.1 - Duplicated Rows

In [ ]:
df_prep = df.copy()
df_prep.set_index('citizen_id', inplace=True)

In [ ]:
# We dropped the duplicate rows
df_prep.drop_duplicates(inplace=True)
df_prep

,name,title,date_of_birth,city,country,last_year_avg_monthly_charity_donations,environmental_awareness_rating,financial_wellness_index,investment_portfolio_value,investments_risk_appetite,investments_risk_tolerance,tech_savviness_score,social_media_influence_score,entertainment_engagement_factor,avg_monthly_entertainment_expenses,avg_weekly_exercise_hours,health_consciousness_rating,stress_management_score,overall_well_being,lifestyle_type
citizen_id,,,,,,,,,,,,,,,,,,,,
1,Stanley,Mr.,1992-01-03,Mining City,Data Land,0.0,1.9352,99.6431,9.7221,3.5313,NaN,11.9772,9.9506,1.75750,23.0605,0.1776,0.6504,2.8850,106.2820,Travel Enthusiast
2,Višeslav,Mr.,2005-03-16,Mining City,Data Land,0.0,0.5672,125.1630,5.4308,6.6175,13.6445,11.8776,5.1521,1.96770,NaN,1.2782,1.0190,3.5777,132.1210,Fitness Enthusiast
3,Rogerio,Mr.,2005-04-04,Mining City,Data Land,1.0,0.5613,105.2490,15.0829,6.9319,5.8867,14.4247,3.6737,3.69071,37.5970,0.0354,3.0854,5.1906,115.5470,Health-Conscious
4,Jagat,Mr.,1981-09-25,Mining City,Data Land,0.0,0.3543,95.2219,16.3380,7.0968,9.1074,12.3668,5.5798,1.57690,8.2142,1.7763,0.3677,NaN,101.7730,Fitness Enthusiast
5,Virgínio,Mr.,1987-07-29,Mining City,Data Land,0.0,0.5943,255.8600,148.2250,5.8014,4.0892,15.8389,3.7712,1.95580,14.3443,0.0000,0.6718,2.7076,264.4510,Adventure Seeker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90161,Salomé,Miss,1993-11-18,Mining City,Data Land,0.0,0.8188,288.7790,0.5000,7.1792,9.5441,12.9497,5.0265,2.16390,44.7958,NaN,1.7052,NaN,296.9590,Investor
90162,امیر,Mr.,1982-02-02,Mining City,Data Land,0.0,0.6033,15.6433,20.1930,5.0231,8.9811,12.5240,5.0200,2.31650,95.1422,0.1997,0.7403,5.0756,22.6456,Investor
90163,Murat,Mr.,1986-01-07,Mining City,Data Land,0.0,0.6710,96.3212,13.1624,6.3267,5.1733,13.4334,12.9103,2.42090,48.7617,2.2265,2.4052,2.9948,105.4430,Fitness Enthusiast


In [ ]:
# We readded the variable citizen_id, but it is based on the index position but +1, since in python the position starts at zero but we want id's to start at 1.
df_prep['citizen_id'] = df_prep.index + 1

In [ ]:
#We replaced the index the new calculated citizen_id (At this stage our data frame doesn't include duplicates, and therefore citizen_id are unique for each row)
df_prep.set_index('citizen_id', inplace=True)
df_prep

,name,title,date_of_birth,city,country,last_year_avg_monthly_charity_donations,environmental_awareness_rating,financial_wellness_index,investment_portfolio_value,investments_risk_appetite,investments_risk_tolerance,tech_savviness_score,social_media_influence_score,entertainment_engagement_factor,avg_monthly_entertainment_expenses,avg_weekly_exercise_hours,health_consciousness_rating,stress_management_score,overall_well_being,lifestyle_type
citizen_id,,,,,,,,,,,,,,,,,,,,
2,Stanley,Mr.,1992-01-03,Mining City,Data Land,0.0,1.9352,99.6431,9.7221,3.5313,NaN,11.9772,9.9506,1.75750,23.0605,0.1776,0.6504,2.8850,106.2820,Travel Enthusiast
3,Višeslav,Mr.,2005-03-16,Mining City,Data Land,0.0,0.5672,125.1630,5.4308,6.6175,13.6445,11.8776,5.1521,1.96770,NaN,1.2782,1.0190,3.5777,132.1210,Fitness Enthusiast
4,Rogerio,Mr.,2005-04-04,Mining City,Data Land,1.0,0.5613,105.2490,15.0829,6.9319,5.8867,14.4247,3.6737,3.69071,37.5970,0.0354,3.0854,5.1906,115.5470,Health-Conscious
5,Jagat,Mr.,1981-09-25,Mining City,Data Land,0.0,0.3543,95.2219,16.3380,7.0968,9.1074,12.3668,5.5798,1.57690,8.2142,1.7763,0.3677,NaN,101.7730,Fitness Enthusiast
6,Virgínio,Mr.,1987-07-29,Mining City,Data Land,0.0,0.5943,255.8600,148.2250,5.8014,4.0892,15.8389,3.7712,1.95580,14.3443,0.0000,0.6718,2.7076,264.4510,Adventure Seeker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90162,Salomé,Miss,1993-11-18,Mining City,Data Land,0.0,0.8188,288.7790,0.5000,7.1792,9.5441,12.9497,5.0265,2.16390,44.7958,NaN,1.7052,NaN,296.9590,Investor
90163,امیر,Mr.,1982-02-02,Mining City,Data Land,0.0,0.6033,15.6433,20.1930,5.0231,8.9811,12.5240,5.0200,2.31650,95.1422,0.1997,0.7403,5.0756,22.6456,Investor
90164,Murat,Mr.,1986-01-07,Mining City,Data Land,0.0,0.6710,96.3212,13.1624,6.3267,5.1733,13.4334,12.9103,2.42090,48.7617,2.2265,2.4052,2.9948,105.4430,Fitness Enthusiast


## 2.2 - Modify variables (recodes)


In [ ]:
# We changed the values in the 'title' column, if equal to Mr. change it to 0 otherwise 1, this way we can transform the title column into a colum
# that shows all the female as 1 and therefore can be used as an integer.
df_prep['female'] = df_prep['title'].map(lambda x: 0 if x == 'Mr.' else 1)

# We dropped the 'title' column since it will be replaced by the column female.
df_prep.drop(columns=['title'], inplace=True)




# Convert original date_of_birth variable to date type.
df_prep['date_of_birth'] = pd.to_datetime(df_prep['date_of_birth'])

# Calculate age from date_of_birth
basis_year = 2024
df_prep['age'] = basis_year - df_prep['date_of_birth'].dt.year

# Define the variable for the calculation
age = df_prep['age']





## Aqui, nós entendemos que os valores negativos de doações significavam erros. São uma pequena parcela da base, mas foram convertidos para absoluto
df_prep['last_year_avg_monthly_charity_donations']=df_prep['last_year_avg_monthly_charity_donations'].abs()
df_prep['avg_weekly_exercise_hours'] = df_prep['avg_weekly_exercise_hours'].abs()

## 2.3 - Missing Values

In [ ]:
#df_prep[metric].isna().sum()

## 2.4 - Mean and Standard Deviation

In [ ]:
#st = df_prep[metric].describe().T
#st

## 2.5 - Histograms and Boxplots



In [ ]:
def plot_histograms_boxplots(df,feats):
    # Determine the number of rows needed for subplots
    num_columns = len(df[feats].columns)
    num_rows = num_columns  # Two plots per column (histogram and boxplot)

    # Create a figure with subplots
    fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(10, num_rows * 4))

    # Flatten the axes array for easy iteration
    axes = axes.flatten()

    # Iterate over each column and create a histogram and a boxplot
    for i, column in enumerate(df[feats].columns):
        # Histogram
        df[feats][column].plot(kind='hist', ax=axes[i * 2], title=f'Histogram of {column}')
        axes[i * 2].set_xlabel(column)

        # Boxplot
        df[feats][column].plot(kind='box', ax=axes[i * 2 + 1], title=f'Boxplot of {column}')

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

In [ ]:
#plot_histograms_boxplots(df_prep,metric)

## 2.6 - Skewness and Curtosis

SKEWNESS & KURTOSIS

In [ ]:
kurtosis_prep = calculate_kurtosis(df_prep[metric])
skewness_prep = calculate_skewness(df_prep[metric])

In [ ]:
#kurtosis_prep

In [ ]:
#skewness_prep


## 2.7 - Power Transformation (Yeo-Jonhson Transform - Skw and Krt)

In [ ]:
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer(method='yeo-johnson')

# Fit the PowerTransformer on the DataFrame and transform the data
df_transformed = pd.DataFrame(pt.fit_transform(df_prep[metric]),columns=df_prep[metric].columns)

df_transformed

,last_year_avg_monthly_charity_donations,environmental_awareness_rating,financial_wellness_index,investment_portfolio_value,investments_risk_appetite,investments_risk_tolerance,tech_savviness_score,social_media_influence_score,entertainment_engagement_factor,avg_monthly_entertainment_expenses,avg_weekly_exercise_hours,health_consciousness_rating,stress_management_score,overall_well_being,age,female
0,-0.398736,1.570922,-0.031187,0.033444,-0.317704,NaN,-0.996532,1.014705,-0.264584,0.262802,-0.424395,-0.463679,-0.197721,-0.057801,-0.349966,-0.999024
1,-0.398736,-0.603849,0.461261,-0.469577,0.812322,1.568264,-1.064898,-0.021190,0.350649,NaN,1.000037,0.225367,0.414985,0.445221,-1.733234,-0.999024
2,2.504880,-0.630486,0.080131,0.420311,0.925277,-0.171375,0.654619,-0.549291,5.859559,0.868822,-0.889007,1.631482,1.607403,0.127156,-1.733234,-0.999024
3,-0.398736,-1.851727,-0.120390,0.490862,0.984388,0.704165,-0.730040,0.104490,-0.781733,-0.994015,1.227943,-1.283186,NaN,-0.149904,0.588134,-0.999024
4,-0.398736,-0.486010,2.609360,2.367959,0.517482,-0.848551,1.585981,-0.508815,0.315448,-0.321830,-1.029533,-0.414245,-0.367869,2.607573,0.095783,-0.999024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90155,-0.398736,0.263816,3.088039,-2.000121,1.013893,0.801838,-0.334022,-0.060030,0.937244,1.086067,NaN,0.961219,NaN,3.071736,-0.443699,1.000977
90156,-0.398736,-0.448449,-2.125349,0.677626,0.233866,0.675173,-0.622927,-0.062066,1.401380,2.015725,-0.363875,-0.265183,1.530772,-2.168957,0.508888,-0.999024
90157,-0.398736,-0.188709,-0.098090,0.300104,0.707542,-0.417524,-0.007751,1.417475,1.722767,1.191179,1.363775,1.377432,-0.095343,-0.074830,0.180802,-0.999024
90158,-0.398736,-0.575801,-0.021341,0.484313,-0.625604,-0.634294,-2.212920,1.329185,-0.996068,-0.597894,1.511212,-0.518204,-0.984487,-0.066035,-1.492093,-0.999024


In [ ]:
kurtosis_yj = calculate_kurtosis(df_transformed)
skewness_yj = calculate_skewness(df_transformed)

In [ ]:
#kurtosis_yj



In [ ]:
#skewness_yj


In [ ]:
#plot_histograms_boxplots(df_transformed,metric)

## 2.8 - Normalization


In [ ]:
scaler = StandardScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df_transformed), columns=df_prep[metric].columns)
df_norm.index = df_prep.index
df_norm

,last_year_avg_monthly_charity_donations,environmental_awareness_rating,financial_wellness_index,investment_portfolio_value,investments_risk_appetite,investments_risk_tolerance,tech_savviness_score,social_media_influence_score,entertainment_engagement_factor,avg_monthly_entertainment_expenses,avg_weekly_exercise_hours,health_consciousness_rating,stress_management_score,overall_well_being,age,female
citizen_id,,,,,,,,,,,,,,,,
2,-0.398736,1.570922,-0.031187,0.033444,-0.317704,NaN,-0.996532,1.014705,-0.264584,0.262802,-0.424395,-0.463679,-0.197721,-0.057801,-0.349966,-0.999024
3,-0.398736,-0.603849,0.461261,-0.469577,0.812322,1.568264,-1.064898,-0.021190,0.350649,NaN,1.000037,0.225367,0.414985,0.445221,-1.733234,-0.999024
4,2.504880,-0.630486,0.080131,0.420311,0.925277,-0.171375,0.654619,-0.549291,5.859559,0.868822,-0.889007,1.631482,1.607403,0.127156,-1.733234,-0.999024
5,-0.398736,-1.851727,-0.120390,0.490862,0.984388,0.704165,-0.730040,0.104490,-0.781733,-0.994015,1.227943,-1.283186,NaN,-0.149904,0.588134,-0.999024
6,-0.398736,-0.486010,2.609360,2.367959,0.517482,-0.848551,1.585981,-0.508815,0.315448,-0.321830,-1.029533,-0.414245,-0.367869,2.607573,0.095783,-0.999024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90162,-0.398736,0.263816,3.088039,-2.000121,1.013893,0.801838,-0.334022,-0.060030,0.937244,1.086067,NaN,0.961219,NaN,3.071736,-0.443699,1.000977
90163,-0.398736,-0.448449,-2.125349,0.677626,0.233866,0.675173,-0.622927,-0.062066,1.401380,2.015725,-0.363875,-0.265183,1.530772,-2.168957,0.508888,-0.999024
90164,-0.398736,-0.188709,-0.098090,0.300104,0.707542,-0.417524,-0.007751,1.417475,1.722767,1.191179,1.363775,1.377432,-0.095343,-0.074830,0.180802,-0.999024


### 2.8.1 - KNN Imputer

In [ ]:
#O df_im vai possuir somente as medidas. Não haverá variáveis categóricas.
#Esse bloco demora muito para correr. -> É normal
imputer = KNNImputer(n_neighbors=4)
df_im = pd.DataFrame(imputer.fit_transform(df_norm[metric]))
df_im.columns = df_prep[metric].columns
df_im.index = df_prep[metric].index
df_im.isna().sum()

last_year_avg_monthly_charity_donations    0
environmental_awareness_rating             0
financial_wellness_index                   0
investment_portfolio_value                 0
investments_risk_appetite                  0
investments_risk_tolerance                 0
tech_savviness_score                       0
social_media_influence_score               0
entertainment_engagement_factor            0
avg_monthly_entertainment_expenses         0
avg_weekly_exercise_hours                  0
health_consciousness_rating                0
stress_management_score                    0
overall_well_being                         0
age                                        0
female                                     0
dtype: int64

In [ ]:
kurtosis_norm = calculate_kurtosis(df_norm)
skewness_norm = calculate_skewness(df_norm)

In [ ]:
kurtosis_norm
# Executive Summary: Kurtosis Interpretation

# Leptokurtic Variables (Kurtosis > 3):
# - last_year_avg_monthly_charity_donations, financial_wellness_index,
#   overall_well_being, entertainment_engagement_factor:
#   Distributions have heavier tails and sharper peaks than a normal distribution.

# Mesokurtic Variables (Kurtosis ≈ 3):
# - environmental_awareness_rating, investment_portfolio_value,
#   health_consciousness_rating, stress_management_score, age:
#   Distributions are approximately similar to a normal distribution in terms of tail heaviness.

# Platykurtic Variables (Kurtosis < 3):
# - investments_risk_appetite, social_media_influence_score,
#   avg_monthly_entertainment_expenses, avg_weekly_exercise_hours, female:
#   Distributions have lighter tails and a flatter peak than a normal distribution.


{'last_year_avg_monthly_charity_donations': 2.4489198184156806,
 'environmental_awareness_rating': 0.26358080946885876,
 'financial_wellness_index': 3.531584402877674,
 'investment_portfolio_value': 0.6189861911210741,
 'investments_risk_appetite': -1.2423549998321346,
 'investments_risk_tolerance': 1.881279362517136,
 'tech_savviness_score': 1.168548927440575,
 'social_media_influence_score': -0.4326613235632246,
 'entertainment_engagement_factor': 2.1032523412816997,
 'avg_monthly_entertainment_expenses': -1.271551888789518,
 'avg_weekly_exercise_hours': -1.475244098235805,
 'health_consciousness_rating': 0.7296585488982652,
 'stress_management_score': 0.39560429784911966,
 'overall_well_being': 3.3635838250231616,
 'age': 0.9648775707783046,
 'female': -2.00004055619707}

In [ ]:
skewness_norm
# Executive Summary: Skewness Interpretation

# Highly Positively Skewed Variables (Skewness > 1):
# - last_year_avg_monthly_charity_donations, avg_weekly_exercise_hours:
#   Distributions are highly skewed to the right.

# Moderately Positively Skewed Variables (0.5 < Skewness < 1):
# - overall_well_being: Distribution is moderately skewed to the right.

# Mildly Positively Skewed Variables (0 < Skewness < 0.5):
# - environmental_awareness_rating, financial_wellness_index,
#   investments_risk_tolerance, tech_savviness_score,
#   entertainment_engagement_factor, avg_monthly_entertainment_expenses,
#   health_consciousness_rating, stress_management_score, age:
#   Distributions are mildly skewed to the right.

# Close to Symmetric Variables (Skewness ≈ 0):
# - investment_portfolio_value, social_media_influence_score, female:
#   Distributions are approximately symmetric.
# - social_media_influence_score, female: Distributions are approximately symmetric.


{'last_year_avg_monthly_charity_donations': 2.109228432214257,
 'environmental_awareness_rating': 0.2529208201530578,
 'financial_wellness_index': 0.3120366733692493,
 'investment_portfolio_value': -0.01530703309925802,
 'investments_risk_appetite': -0.30739836676848287,
 'investments_risk_tolerance': 0.01589186671422224,
 'tech_savviness_score': 0.02656836642393895,
 'social_media_influence_score': 0.013473532159131234,
 'entertainment_engagement_factor': 0.081024899974471,
 'avg_monthly_entertainment_expenses': 0.009545380837322247,
 'avg_weekly_exercise_hours': 0.434264798806514,
 'health_consciousness_rating': 0.004285901792044036,
 'stress_management_score': 0.008499756826758893,
 'overall_well_being': 0.2837257979242732,
 'age': 0.07011950883189072,
 'female': 0.0019521185894574153}

In [ ]:
#tests for normality of the attributes
from scipy.stats import kstest
for column in df_norm.columns:
    data = df_norm[column]  # Extract the data from the DataFrame column
    statistic, p_value = kstest(data, 'norm')  # Perform Kolmogorov-Smirnov test against a normal distribution
    print(f"Variable: {column}")
    print(f"Kolmogorov-Smirnov Test - Statistic: {statistic}, p-value: {p_value}")

    # Interpret the test result
    alpha = 0.05
    if p_value > alpha:
        print("Sample looks Gaussian (fail to reject H0)")
    else:
        print("Sample does not look Gaussian (reject H0)")
    print("\n")


Variable: last_year_avg_monthly_charity_donations
Kolmogorov-Smirnov Test - Statistic: nan, p-value: nan
Sample does not look Gaussian (reject H0)


Variable: environmental_awareness_rating
Kolmogorov-Smirnov Test - Statistic: nan, p-value: nan
Sample does not look Gaussian (reject H0)


Variable: financial_wellness_index
Kolmogorov-Smirnov Test - Statistic: nan, p-value: nan
Sample does not look Gaussian (reject H0)


Variable: investment_portfolio_value
Kolmogorov-Smirnov Test - Statistic: nan, p-value: nan
Sample does not look Gaussian (reject H0)


Variable: investments_risk_appetite
Kolmogorov-Smirnov Test - Statistic: nan, p-value: nan
Sample does not look Gaussian (reject H0)


Variable: investments_risk_tolerance
Kolmogorov-Smirnov Test - Statistic: nan, p-value: nan
Sample does not look Gaussian (reject H0)


Variable: tech_savviness_score
Kolmogorov-Smirnov Test - Statistic: nan, p-value: nan
Sample does not look Gaussian (reject H0)


Variable: social_media_influence_score
K

In [ ]:
#tests for normality of the attributes
from scipy.stats import anderson
for column in df_norm.columns:
    data = df_norm[column]  # Extract the data from the DataFrame column
    result = anderson(data)  # Perform Anderson-Darling test
    statistic = result.statistic
    critical_values = result.critical_values
    significance_level = result.significance_level

    print(f"Variable: {column}")
    print(f"Anderson-Darling Test - Statistic: {statistic}")
    print(f"Critical Values: {critical_values}")
    print(f"Significance Level: {significance_level}")

    # Interpret the test result
    alpha = 0.05
    if statistic < critical_values[2]:
        print("Sample looks Gaussian (fail to reject H0)")
    else:
        print("Sample does not look Gaussian (reject H0)")
    print("\n")


Variable: last_year_avg_monthly_charity_donations
Anderson-Darling Test - Statistic: nan
Critical Values: [0.576 0.656 0.787 0.918 1.092]
Significance Level: [15.  10.   5.   2.5  1. ]
Sample does not look Gaussian (reject H0)


Variable: environmental_awareness_rating
Anderson-Darling Test - Statistic: nan
Critical Values: [0.576 0.656 0.787 0.918 1.092]
Significance Level: [15.  10.   5.   2.5  1. ]
Sample does not look Gaussian (reject H0)


Variable: financial_wellness_index
Anderson-Darling Test - Statistic: nan
Critical Values: [0.576 0.656 0.787 0.918 1.092]
Significance Level: [15.  10.   5.   2.5  1. ]
Sample does not look Gaussian (reject H0)


Variable: investment_portfolio_value
Anderson-Darling Test - Statistic: nan
Critical Values: [0.576 0.656 0.787 0.918 1.092]
Significance Level: [15.  10.   5.   2.5  1. ]
Sample does not look Gaussian (reject H0)


Variable: investments_risk_appetite
Anderson-Darling Test - Statistic: nan
Critical Values: [0.576 0.656 0.787 0.918 1.09

## 2.9 - DBScan - Outliers






In [ ]:
def plot_kdist_graph(df, feats, n_neighbors):
    """
    Plots the K-distance graph to determine the right epsilon (eps) value for DBSCAN clustering.

    Parameters:
    - df: DataFrame containing the data points.
    - feats: List of feature columns to consider for distance calculation.
    - n_neighbors: Number of nearest neighbors to consider for average distance calculation.

    This function calculates the average distance to the n_neighbors for each data point
    and plots the sorted distances to help identify an appropriate epsilon value for DBSCAN clustering.

    Args:
        df (DataFrame): The data frame containing the data points.
        feats (list): List of feature columns to consider for distance calculation.
        n_neighbors (int): Number of nearest neighbors to consider for average distance calculation.

    Returns:
        None
    """
    neigh = NearestNeighbors(n_neighbors=n_neighbors)
    neigh.fit(df[feats])
    distances, _ = neigh.kneighbors(df[feats])

    # We sort the average distances of the points and plot this
    distances = np.sort(distances[:, -1])
    plt.ylabel("%d-NN Distance" % n_neighbors)
    plt.xlabel("Points sorted by distance")
    plt.plot(distances)
    plt.title("DBScan - Kdist")
    plt.show()

In [ ]:
## Esse aqui demora muito também
#plot_kdist_graph(df_im, metric,n_neighbors = 10)

In [ ]:
## Create a DBSCAN clustering model with specified hyperparameters
dbscan = DBSCAN(eps=2.5, min_samples=10, n_jobs=4)

## Fit the DBSCAN model to the data and predict cluster labels
dbscan_labels = dbscan.fit_predict(df_im[metric])

## Print information about the clustering results
print("Points in cluster -1 are noise rows.")
print("Counter of cluster labels     :", Counter(dbscan_labels))
print("Percentage of noise rows      :", round(100*Counter(dbscan_labels)[-1]/df.shape[0],2))

Points in cluster -1 are noise rows.
Counter of cluster labels     : Counter({0: 89198, -1: 962})
Percentage of noise rows      : 1.07


In [ ]:
def split_noise_rows(df, feats, dbs_model):
  ## Predict cluster labels using the provided DBSCAN model
  dbscan_labels = dbs_model.fit_predict(df[feats])

  ## Concatenate cluster labels with the original DataFrame
  df_concat = pd.concat([df,
                         pd.Series(dbscan_labels,
                                   name='dbscan_labels',
                                   index=df.index)],
                            axis=1)

  ## Create separate DataFrames for noise and non-noise rows
  df_noise = df_concat[df_concat['dbscan_labels']==-1].copy()
  df_nonoise = df_concat[df_concat['dbscan_labels']== 0].copy()

  return df_noise, df_nonoise, df_concat

In [ ]:
df_im_with_target = df_im.copy()
#Reintegrate target to implement RFE
df_im_with_target['lifestyle_type'] = df_prep['lifestyle_type']

In [ ]:
df_noise, df_nonoise, df_concat = split_noise_rows(df_im_with_target, metric, dbscan)

# 3 - Test Set Preparation

### 3.1 - No Duplicated Rows

## 3.2 - Replace Index

In [ ]:
dft_prep = dft.copy()
dft_prep.set_index('citizen_id', inplace=True)
dft_prep.shape

(38643, 19)

## 3.3 - Modify variables

In [ ]:
#Title to female
dft_prep['female'] = dft_prep['title'].map(lambda x: 0 if x == 'Mr.' else 1)
dft_prep.drop(columns=['title'], inplace=True)

In [ ]:
# Convert original date_of_birth variable to date type.
dft_prep['date_of_birth'] = pd.to_datetime(dft_prep['date_of_birth'])

# Calculate age from date_of_birth, fixing 2024 as year.
basis_year = 2024
dft_prep['age'] = basis_year - dft_prep['date_of_birth'].dt.year

# Define the variable for the calculation
age = dft_prep['age']

In [ ]:
## Aqui, nós entendemos que os valores negativos de doações significavam erros. São uma pequena parcela da base, mas foram convertidos para absoluto
dft_prep['last_year_avg_monthly_charity_donations']=dft_prep['last_year_avg_monthly_charity_donations'].abs()
dft_prep['avg_weekly_exercise_hours'] = dft_prep['avg_weekly_exercise_hours'].abs()

## 3.4 - Missing Values

In [ ]:
dft_prep.isna().sum()

name                                          0
date_of_birth                                 0
city                                          0
country                                       0
last_year_avg_monthly_charity_donations    1353
environmental_awareness_rating              966
financial_wellness_index                    966
investment_portfolio_value                 1159
investments_risk_appetite                   580
investments_risk_tolerance                  580
tech_savviness_score                        966
social_media_influence_score               1353
entertainment_engagement_factor            1353
avg_monthly_entertainment_expenses          580
avg_weekly_exercise_hours                  1546
health_consciousness_rating                 966
stress_management_score                    1546
overall_well_being                          580
female                                        0
age                                           0
dtype: int64

## 3.5 - Histograms and Boxplots

In [ ]:
#plot_histograms_boxplots(dft_prep,metric)

## 3.6 - Skewness and Kurtosis

In [ ]:
kr_test = calculate_kurtosis(dft_prep[metric])
sk_test = calculate_skewness(dft_prep[metric])

In [ ]:
#kr_test

In [ ]:
#sk_test

## 3.7 - Power Transformation

In [ ]:
pt = PowerTransformer(method='yeo-johnson')

# Fit the PowerTransformer on the DataFrame and transform the data
dft_transformed = pd.DataFrame(pt.fit_transform(dft_prep[metric]),columns=dft_prep[metric].columns)

dft_transformed

,last_year_avg_monthly_charity_donations,environmental_awareness_rating,financial_wellness_index,investment_portfolio_value,investments_risk_appetite,investments_risk_tolerance,tech_savviness_score,social_media_influence_score,entertainment_engagement_factor,avg_monthly_entertainment_expenses,avg_weekly_exercise_hours,health_consciousness_rating,stress_management_score,overall_well_being,age,female
0,NaN,-0.193770,-0.013013,0.067643,-0.989992,0.534814,1.143435,0.920536,1.200037,1.529393,-0.792140,-0.211288,1.440491,-0.003291,0.165959,1.005579
1,-0.398354,-0.473950,0.034167,-0.236436,-1.696687,0.058950,1.289437,0.370262,-0.055403,0.016895,0.392184,0.686132,-0.545789,0.059108,1.607097,1.005579
2,-0.398354,-0.211190,0.036013,0.518746,-0.765613,-0.771068,-0.485580,1.289653,-0.219727,-0.611928,1.047779,1.428501,-0.458458,0.057217,-0.013096,1.005579
3,-0.398354,1.723794,-0.000906,0.162704,1.006382,1.663679,-1.316354,-0.560768,0.779100,1.187461,1.462706,-0.132835,0.158507,-0.027095,1.132905,-0.994452
4,-0.398354,-0.464842,-0.007456,-0.575295,-0.048217,-0.040271,1.604771,0.641449,-1.011778,0.658823,-1.033544,0.242639,-0.949555,0.014873,-0.620071,-0.994452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38638,-0.398354,1.925355,-0.498054,1.881739,1.265580,1.313436,0.531958,-3.237365,-2.610891,-1.338480,0.604799,-0.368661,-3.143742,-0.503324,0.573724,1.005579
38639,-0.398354,-1.449620,0.050524,-0.131414,-1.468446,-1.136797,-0.270898,1.422281,2.126364,1.160588,-0.691432,-0.687135,NaN,0.035122,0.416879,-0.994452
38640,-0.398354,0.932666,0.137432,2.141695,0.850903,1.065118,-1.788176,-1.665466,-0.444141,-0.250415,1.042168,0.132667,-1.071479,0.109118,-0.202857,-0.994452
38641,NaN,-0.372248,-0.000586,0.237003,1.086535,0.435944,-1.368978,NaN,-0.318086,0.837471,-1.033544,0.495024,-0.157203,-0.019029,-0.106561,-0.994452


## 3.8 - Normalization

In [ ]:
scaler = StandardScaler()
dft_norm = pd.DataFrame(scaler.fit_transform(dft_transformed), columns=dft_prep[metric].columns)
dft_norm.index = dft_prep.index
dft_norm

,last_year_avg_monthly_charity_donations,environmental_awareness_rating,financial_wellness_index,investment_portfolio_value,investments_risk_appetite,investments_risk_tolerance,tech_savviness_score,social_media_influence_score,entertainment_engagement_factor,avg_monthly_entertainment_expenses,avg_weekly_exercise_hours,health_consciousness_rating,stress_management_score,overall_well_being,age,female
citizen_id,,,,,,,,,,,,,,,,
90166,NaN,-0.193770,-0.013013,0.067643,-0.989992,0.534814,1.143435,0.920536,1.200037,1.529393,-0.792140,-0.211288,1.440491,-0.003291,0.165959,1.005579
90167,-0.398354,-0.473950,0.034167,-0.236436,-1.696687,0.058950,1.289437,0.370262,-0.055403,0.016895,0.392184,0.686132,-0.545789,0.059108,1.607097,1.005579
90168,-0.398354,-0.211190,0.036013,0.518746,-0.765613,-0.771068,-0.485580,1.289653,-0.219727,-0.611928,1.047779,1.428501,-0.458458,0.057217,-0.013096,1.005579
90169,-0.398354,1.723794,-0.000906,0.162704,1.006382,1.663679,-1.316354,-0.560768,0.779100,1.187461,1.462706,-0.132835,0.158507,-0.027095,1.132905,-0.994452
90170,-0.398354,-0.464842,-0.007456,-0.575295,-0.048217,-0.040271,1.604771,0.641449,-1.011778,0.658823,-1.033544,0.242639,-0.949555,0.014873,-0.620071,-0.994452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128804,-0.398354,1.925355,-0.498054,1.881739,1.265580,1.313436,0.531958,-3.237365,-2.610891,-1.338480,0.604799,-0.368661,-3.143742,-0.503324,0.573724,1.005579
128805,-0.398354,-1.449620,0.050524,-0.131414,-1.468446,-1.136797,-0.270898,1.422281,2.126364,1.160588,-0.691432,-0.687135,NaN,0.035122,0.416879,-0.994452
128806,-0.398354,0.932666,0.137432,2.141695,0.850903,1.065118,-1.788176,-1.665466,-0.444141,-0.250415,1.042168,0.132667,-1.071479,0.109118,-0.202857,-0.994452


## 3.9 - KNN Imputer


In [ ]:
#O df_im vai possuir somente as medidas. Não haverá variáveis categóricas.
#Esse bloco demora muito para correr. -> É normal
imputer = KNNImputer(n_neighbors=4)
dft_im = pd.DataFrame(imputer.fit_transform(dft_norm[metric]))
## Set correct column names and index
dft_im.columns = dft_prep[metric].columns
dft_im.index = dft_prep[metric].index

#Valida imputer
#dft_im.isna().sum()

# 4 - Feature Selection

### 4.1 - Spearman Correlation

In [ ]:

def plot_corr_matrix(df, metric_variables, title="Correlation Matrix", method = "spearman", figsize=(10,8)):

  # Prepare figure
  figsize = (10*2, 8*2)
  fig = plt.figure(figsize=figsize)

  ## Calculate the correlation matrix for the specified metric variables rounded to 2 decimal places
  corr = np.round(df[metric_variables].corr(method=method), decimals=2)

  ## Create an annotation matrix to selectively display highly correlated values (>|0.5|)
  mask_annot = np.absolute(corr.values) >= 0.5
  ## Use np.where() to conditionally fill matrix elements for annotation display
  annot = np.where(mask_annot, corr.values, np.full(corr.shape,"")) # Try to understand what this np.where() does

  ## Plot the heatmap representing the correlation matrix
  sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True),
              fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

  ## Adjust the layout for better visualization
  fig.subplots_adjust(top=0.95)
  fig.suptitle(title, fontsize=20)

  ## Display the plot
  plt.show()


  return

In [ ]:
#plot_corr_matrix(df_nonoise, metric)

Based on Spearman correlation, we have excluded variables entertainment_engagement_factor and overall_well_being.
We have made this decision because entertainment engagement is highly correlated with two variables (stress management and avg_monthly entertainment) and overall_well_being is more difficult to explain/measure in a realistic framework.

In [ ]:
metric_new = ['last_year_avg_monthly_charity_donations',
'environmental_awareness_rating',
'financial_wellness_index',
'investment_portfolio_value',
'investments_risk_appetite',
'investments_risk_tolerance',
'tech_savviness_score',
'social_media_influence_score',
#'entertainment_engagement_factor',
'avg_monthly_entertainment_expenses',
'avg_weekly_exercise_hours',
'health_consciousness_rating',
'stress_management_score',
#'overall_well_being',
'age',
'female']



len(metric_new)

14

### 4.2 - RFE

In [ ]:
# Define train data
X_train = df_nonoise[metric_new]
Y_train = df_nonoise['lifestyle_type']

In [ ]:

#model = RandomForestClassifier(n_jobs=14)


#param_grid = {
#    'n_features_to_select': np.arange(1, 14)
#}


#rfe = RFE(estimator=model)

# Define GridSearchCV
#grid_search = GridSearchCV(estimator=rfe, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=14)

# Fit the GridSearchCV
#grid_search.fit(X_train, Y_train)

# Get the best parameters and best score
#best_params = grid_search.best_params_
#best_score = grid_search.best_score_

#print("Best number of features to select: ", best_params['n_features_to_select'])
#print("Best cross-validation score: %f" % best_score)


In [ ]:
## PUT HERE THE PARAMETERS NUMBER

In [ ]:
# Use RFE to select the optimal number of features (11)
#rfe = RFE(estimator=model, n_features_to_select=11)

# Fit and transform the training data
#rfe_fitted = rfe.fit_transform(X_train, Y_train)

# Get the support mask of the selected features
#selected_features_mask = rfe.support_

# Create a series with the selected feature names
#selected_features = pd.Series(selected_features_mask, index=X_train.columns)
#selected_feature_names = selected_features[selected_features].index.tolist()

# Print the selected feature names
#print("Selected feature names: ", selected_feature_names)

In [ ]:
selected_feature_names = ['environmental_awareness_rating',
 'financial_wellness_index',
   'investment_portfolio_value',
     'investments_risk_appetite',
       'investments_risk_tolerance',
         'tech_savviness_score',
           'social_media_influence_score',
             'avg_monthly_entertainment_expenses',
               'avg_weekly_exercise_hours',
                 'health_consciousness_rating',
                   'stress_management_score']

#### 4.3 - Final Features

In [ ]:
final_features = selected_feature_names
final_features

['environmental_awareness_rating',
 'financial_wellness_index',
 'investment_portfolio_value',
 'investments_risk_appetite',
 'investments_risk_tolerance',
 'tech_savviness_score',
 'social_media_influence_score',
 'avg_monthly_entertainment_expenses',
 'avg_weekly_exercise_hours',
 'health_consciousness_rating',
 'stress_management_score']

# 5 - Final Datasets for Model Training

In [ ]:
# Final Train Set
X_train = df_nonoise[final_features]

#Final test/submission set.
X_test = dft_im[final_features]

#Train and validation sets. Validation set is a subset of train set. -> for troubleshooting purposes
R_train, R_val, RY_train, RY_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# 6 - Models

### 6.0 - Evaluation Functions

In [ ]:
def model_eval(model, train, test, target_train, target_test):
    ft = model.fit(train,target_train)
    pred = ft.predict(train)
    predt = ft.predict(test)

    train_f1 = f1_score(target_train, pred, average='weighted')
    test_f1 = f1_score(target_test, predt, average='weighted')

    evaluation = {
        'F1 Score': [train_f1, test_f1]
    }

    return evaluation

### 6.1 - One-Vs-Rest Logistic Regression

One-vs-Rest Approach: Since we have multiple categories (more than two). This means building separate models for each lifestyle type, comparing it to all others.

In [ ]:
#param_grid = {
#    'penalty': ['l1', 'l2'],
#    'C': [0.01, 0.1, 1, 10, 100],
#   'l1_ratio': [0, 0.25, 0.5, 0.75, 1]
#}


#algrthm = LogisticRegression(multi_class='ovr',random_state= 1 )


#clf = GridSearchCV(algrthm, param_grid, cv=5, scoring='f1_micro')
#clf.fit(X_train, Y_train)

#results = clf.cv_results_

#params = results['params']
#mean_test_scores = results['mean_test_score']


 #Convert the results to a DataFrame
#results_df = pd.DataFrame(results)

# Sort the DataFrame by mean test score in descending order
#sorted_results_df = results_df.sort_values(by='mean_test_score', ascending=False)

# Display the best parameters and corresponding score
#best_result = sorted_results_df.iloc[0]
#LogReg = sorted_results_df[['params','mean_test_score','mean_fit_time']]

In [ ]:
#LogReg.head()

### 6.2 - Multinomial Logistic Regression

In [ ]:
# Initialize the Logistic Regression model with multinomial logistic regression
#param_grid = {
#    'penalty': ['l1', 'l2', 'elasticnet'],
#    'C': [0.01, 0.1, 1, 10, 100],
#   'l1_ratio': [0, 0.25, 0.5, 0.75, 1]  # Only used if penalty='elasticnet'
#}


#algrthm = LogisticRegression(multi_class='multinomial', random_state= 1 )


#clf = GridSearchCV(algrthm, param_grid, cv=5, scoring='f1_micro')
#clf.fit(X_train, Y_train)

#results = clf.cv_results_

#params = results['params']
#mean_test_scores = results['mean_test_score']


 #Convert the results to a DataFrame
#results_df = pd.DataFrame(results)

# Sort the DataFrame by mean test score in descending order
#sorted_results_df = results_df.sort_values(by='mean_test_score', ascending=False)

# Display the best parameters and corresponding score
#best_result = sorted_results_df.iloc[0]
#MultinomialReg = sorted_results_df[['params','mean_test_score','mean_fit_time']]

In [ ]:

#MultinomialReg.head()

### 6.3 - Decision Tree

In [ ]:
# Example usage

#param_grid = {
#    'criterion': ['gini', 'entropy'],
#    'max_depth': [50,100,200,300],
#    'min_samples_split': [20,30,40],
#    'min_samples_leaf': [20,30,40],
#    'max_features': ['sqrt', 'log2'],
#    'max_leaf_nodes': [50, 100, 200],
#}

#algrthm = DecisionTreeClassifier(random_state=1)


#clf = GridSearchCV(algrthm, param_grid, cv=5, scoring='f1_micro')
#clf.fit(X_train, Y_train)

#results = clf.cv_results_

#params = results['params']
#mean_test_scores = results['mean_test_score']


 #Convert the results to a DataFrame
#results_df = pd.DataFrame(results)

# Sort the DataFrame by mean test score in descending order
#sorted_results_df = results_df.sort_values(by='mean_test_score', ascending=False)

# Display the best parameters and corresponding score
#best_result = sorted_results_df.iloc[0]
#DecisionTree = sorted_results_df[['params','mean_test_score','mean_fit_time']]


In [ ]:
#DecisionTree.head()

### 6.4 - Random Forest

In [ ]:
#param_grid = {
    #'max_depth': [200, 500],
    #'n_estimators': [500, 1000],
    #'max_features': ['sqrt', 'log2']
#}
#algrthm = RandomForestClassifier(n_jobs = 18, random_state= 1)


#clf = GridSearchCV(algrthm, param_grid, cv=5, scoring='f1_micro')
#clf.fit(X_train, Y_train)

#results = clf.cv_results_

#params = results['params']
#mean_test_scores = results['mean_test_score']


 #Convert the results to a DataFrame
#results_df = pd.DataFrame(results)

# Sort the DataFrame by mean test score in descending order
#sorted_results_df = results_df.sort_values(by='mean_test_score', ascending=False)

# Display the best parameters and corresponding score
#best_result = sorted_results_df.iloc[0]
#RndFrst = sorted_results_df[['params','mean_test_score','mean_fit_time']]

In [ ]:
#RndFrst.head()

In [ ]:
#rf = RandomForestClassifier(n_jobs=14, max_depth = 50, n_estimators= 800, random_state= 1, max_features= 'sqrt')
#rf.fit(R_train,RY_train)
#model_eval(rf,R_train,R_val,RY_train,RY_val)

### 6.5 - Neural Network

In [ ]:


#param_grid = {
#    'hidden_layer_sizes': [(200,100,50,30),(200,100), (200,100,50)],
#    'activation': ['relu'],
#    'solver': ['adam'],
#    'alpha': [0.1],
#    'learning_rate': ['adaptive'],
#    'learning_rate_init': [0.001],
#   'max_iter': [100,200,300],
#    'n_iter_no_change':[10]
#}

#algrthm = MLPClassifier(random_state= 1)


#clf = GridSearchCV(algrthm, param_grid, cv=5, scoring='f1_micro')
#clf.fit(X_train, Y_train)

#results = clf.cv_results_

#params = results['params']
#mean_test_scores = results['mean_test_score']


 #Convert the results to a DataFrame
#results_df = pd.DataFrame(results)

# Sort the DataFrame by mean test score in descending order
#sorted_results_df = results_df.sort_values(by='mean_test_score', ascending=False)

# Display the best parameters and corresponding score
#best_result = sorted_results_df.iloc[0]
#MLPC = sorted_results_df[['params','mean_test_score','mean_fit_time']]





## 6.6 - Ada Boost

In [ ]:
#param_grid = {
#    'n_estimators': [100, 200 , 500, 700],
#    'learning_rate': [0.001 ,0.01, 0.1, 1.0],
#    'algorithm': ['SAMME', 'SAMME.R']
#}

#algrthm = AdaBoostClassifier(random_state=1)


#clf = GridSearchCV(algrthm, param_grid, cv=5, scoring='f1_micro')
#clf.fit(X_train, Y_train)

#results = clf.cv_results_

#params = results['params']
#mean_test_scores = results['mean_test_score']


 #Convert the results to a DataFrame
#results_df = pd.DataFrame(results)

# Sort the DataFrame by mean test score in descending order
#sorted_results_df = results_df.sort_values(by='mean_test_score', ascending=False)

# Display the best parameters and corresponding score
#best_result = sorted_results_df.iloc[0]
#ADBC = sorted_results_df[['params', 'mean_test_score', 'mean_fit_time']]

#ADBC



In [ ]:
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_columns', None)  # Show all columns
#pd.set_option('display.width', None)  # Set the display width to None
#pd.set_option('display.max_colwidth', None)  # Set the maximum column width to None
#ADBC.head

# 7 - Final Models

In [ ]:
#Submission 09
#Nn3 =  MLPClassifier(hidden_layer_sizes= (200,100,50,25,15,10,5),
#                   learning_rate='constant',alpha=0.1, random_state = 1, activation= 'relu',
#                     max_iter=300, solver = 'adam', verbose=True, tol = 0.0005
#                     )
#Nn3.fit(X_train,Y_train)

In [ ]:
#Submission 10 - BEST MODEL UNTIL 09/06/2024 00:35
Nn2= MLPClassifier(hidden_layer_sizes= (200,100,50), learning_rate='adaptive',alpha=0.1, random_state = 1, activation= 'relu', max_iter=300, solver = 'adam', verbose=True)
Nn2.fit(X_train,Y_train)


Iteration 1, loss = 0.77312010
Iteration 2, loss = 0.65359844
Iteration 3, loss = 0.62468145
Iteration 4, loss = 0.60642038
Iteration 5, loss = 0.59581169
Iteration 6, loss = 0.58586393
Iteration 7, loss = 0.57941264
Iteration 8, loss = 0.57429694
Iteration 9, loss = 0.56796334
Iteration 10, loss = 0.56370389
Iteration 11, loss = 0.56095949
Iteration 12, loss = 0.55835841
Iteration 13, loss = 0.55641133
Iteration 14, loss = 0.55374992
Iteration 15, loss = 0.55137528
Iteration 16, loss = 0.54857502
Iteration 17, loss = 0.54770542
Iteration 18, loss = 0.54536655
Iteration 19, loss = 0.54310982
Iteration 20, loss = 0.54176411
Iteration 21, loss = 0.54110830
Iteration 22, loss = 0.54067058
Iteration 23, loss = 0.53820654
Iteration 24, loss = 0.53725942
Iteration 25, loss = 0.53571720
Iteration 26, loss = 0.53453579
Iteration 27, loss = 0.53526137
Iteration 28, loss = 0.53291112
Iteration 29, loss = 0.53189370
Iteration 30, loss = 0.53158179


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(alpha=0.1, hidden_layer_sizes=(200, 100, 50),
              learning_rate='adaptive', max_iter=300, random_state=1,
              verbose=True)

In [ ]:
#model_eval(model, train, test, target_train, target_test):
#model = Nn2

#model_eval(model,R_train,R_val,RY_train, RY_val)

In [ ]:
#BEST MODEL UNTIL 08/06/2024 23:49 - submisson 05
#Nn1= MLPClassifier(hidden_layer_sizes= (100,50,30,20),learning_rate='adaptive',alpha=0.1, random_state = 1)
#Nn1.fit(X_train,Y_train)


In [ ]:
#Nn= MLPClassifier(hidden_layer_sizes= (100,50),learning_rate='adaptive',alpha=0.1)
#Nn.fit(X_train,Y_train)


### Outputs

In [ ]:
 #predictions = Nn2.predict(X_test[final_features])

#predictions_df = pd.DataFrame({'citizen_id': X_test.index, 'lifestyle_type': predictions})

 #Seting the destination directory
#destination_directory = ''

 #Saving the DataFrame to a CSV file in the specified directory
#predictions_df.to_csv(os.path.join(destination_directory, 'NT_Group17__Version.csv'), index=False)